In [1]:
#### 1 - List of nominated movies to the following categories :
# Best Director (1927-2021)
# Best Actor (1927-2021)
# Best Actress (1927-2021)

import requests
from scrapy import Selector

list_response = [] ## LIST OF EVERY SELECTOR (scrapy) for every year of OSCARS

for i in range(1930,2022): # from 1930 (first Oscars to 2021 Oscars)
    time.sleep(10) # Crawlerdelay
    url = "https://www.oscars.org/oscars/ceremonies/{}".format(i)
    response_oscars = requests.get(url)
    sel = Selector(text = response_oscars.text)   # Scrapy chose over BeautifulSoup for selector CSS
    list_response.append(sel)
    



[<Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Sele

In [3]:
### OOP   ### IN PROGRESS TO GO FROM PROCEDURAL TO OOP

class Oscar_Scaper:
    def __init__(self, time_period):
      self.links = {
          'pages':[],
          'something':[]
      }
      self.data = {
          'category':[],
          'film':[]
          'name':[],
          'gender':[],
          'result':[],
          'birthdate':[]
      }
    
    
      





SyntaxError: invalid syntax (<ipython-input-3-befceb31fb84>, line 12)

In [207]:
###IMPORTANT ####
 # Actor until 1976
 # Actor in a Leading Role since 1977
 # Same for Actress
    
### Stock the data better than doing a request every try # Done

### PROBLEME : COMMENT GERER LES 2 WINNERS, existe plusieurs fois # Done

### PROBLEM : for DIRECTION, names and titles are swapped # Done
    



In [189]:
### COMPLETE FOR OSCAR.ORG ALTERNATIVE WITH ONLY LIST_NAMES
    
list_names = []
list_year = []
list_category = []
list_films = []
list_results = []

for years in range(len(list_response)):
    
    categories = list_response[years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
    for i in range(len(categories)):
        if categories[i] in ('Directing', 'Actor','Actor in a Leading Role', 'Actress', 'Actress in a Leading Role'):
            
            # Need the number of winners (might be ties) and nominated          
            path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
            number_of_film_nominated = ''.join(list_response[years].css(path_nominated).extract()).split("views-row-")    
            
            
            path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
            number_of_film_winner = ''.join(list_response[years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
            
            
            
                 ### WINNER(S) ####
            if len(number_of_film_winner) == 2: # 1 Winner
                # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                list_year.append(1930 + years)
                list_category.append(categories[i])
                list_results.append("Winner")
                if categories[i] == "Directing": # For Directing, titles and names are swapped on the website
                    list_films.append(nominee_name)
                else:
                    list_names.append(nominee_name)

                # FIlM
                path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
                nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
                if categories[i] == "Directing":# For Directing, titles and names are swapped on the website
                    list_names.append(nominee_film)
                else:
                    list_films.append(nominee_film)
                
                for number in range(int(number_of_film_nominated[1])):   
                ### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
                    list_year.append(1930 + years)         
                    list_category.append(categories[i])
                    list_results.append("Nominee")

                    if categories[i] == "Directing": # For Directing, titles and names are swapped on the website
                        list_films.append(nominee_name)
                    else:
                        list_names.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
                    if categories[i] == "Directing":# For Directing, titles and names are swapped on the website
                        list_names.append(nominee_film)
                    else:
                        list_films.append(nominee_film)    
                    
            if len(number_of_film_winner) == 1: # 2 Winners
                ### WINNERS ###       
                # NAME 1st Winner 
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_year.append(1930 + years)
                list_category.append(categories[i])
                list_results.append("Tie-winner")

                if categories[i] == "Directing": # For Directing, titles and names are swapped on the website
                    list_films.append(nominee_name)
                else:
                    list_names.append(nominee_name)
                    
                # NAME 2st Winner 
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_year.append(1930 + years)
                list_category.append(categories[i])
                list_results.append("Tie-winner")
                if categories[i] == "Directing": # For Directing, titles and names are swapped on the website
                    list_films.append(nominee_name)
                else:
                    list_names.append(nominee_name)

                
                # FILM
                path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
                path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

                nominee_film = ''.join(list_response[years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
                
                if categories[i] == "Directing":# For Directing, titles and names are swapped on the website
                    list_names.append(nominee_film)
                else:
                    list_films.append(nominee_film)

                winner2_film = ''.join(list_response[years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
                winner2_film = winner2_film.replace("\n", "") # Cleaning up the string
            
                if categories[i] == "Directing":# For Directing, titles and names are swapped on the website
                    list_names.append(nominee_film)
                else:
                    list_films.append(nominee_film)

                for number in range(int(number_of_film_nominated[1])):   
                    #### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 5 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
                    list_year.append(1930 + years)
                    list_category.append(categories[i])
                    list_results.append("Nominee")
                    if categories[i] == "Directing": # For Directing, titles and names are swapped on the website
                        list_films.append(nominee_name)
                    else:
                        list_names.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 5 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

                    if categories[i] == "Directing":# For Directing, titles and names are swapped on the website
                        list_names.append(nominee_film)
                    else:
                        list_films.append(nominee_film)     

     

print("Done")


Done


['Warner Baxter', 'George Bancroft', 'Chester Morris', 'Paul Muni', 'Lewis Stone', 'Mary Pickford', 'Ruth Chatterton', 'Betty Compson', 'Jeanne Eagels', 'Corinne Griffith', 'Bessie Love', 'Frank Lloyd', 'Lionel Barrymore', 'Harry Beaumont', 'Irving Cummings', 'Frank Lloyd', 'Frank Lloyd', 'Ernst Lubitsch', 'George Arliss', 'George Arliss', 'Wallace Beery', 'Maurice Chevalier', 'Maurice Chevalier', 'Ronald Colman', 'Ronald Colman', 'Lawrence Tibbett', 'Norma Shearer', 'Nancy Carroll', 'Ruth Chatterton', 'Greta Garbo', 'Greta Garbo', 'Norma Shearer', 'Gloria Swanson', 'Lewis Milestone', 'Clarence Brown', 'Clarence Brown', 'Robert Leonard', 'Ernst Lubitsch', 'King Vidor', 'Lionel Barrymore', 'Jackie Cooper', 'Richard Dix', 'Fredric March', 'Adolphe Menjou', 'Marie Dressler', 'Marlene Dietrich', 'Irene Dunne', 'Ann Harding', 'Norma Shearer', 'Norman Taurog', 'Wesley Ruggles', 'Clarence Brown', 'Lewis Milestone', 'Josef Von Sternberg', 'Wallace Beery', 'Fredric March', 'Alfred Lunt', 'Helen

In [199]:
# importing pandas as pd 
import pandas as pd 

  
# dictionary of lists 
dict = {'name': list_names, 'category': list_category, 'film': list_films, 'result': list_results, 'year': list_year} 
    
df = pd.DataFrame(dict)
df = df.set_index(['year','category'])
df 

name                   film   result
year category                                                    
1930 Actor          Warner Baxter         In Old Arizona   Winner
     Actor        George Bancroft            Thunderbolt  Nominee
     Actor         Chester Morris                  Alibi  Nominee
     Actor              Paul Muni            The Valiant  Nominee
     Actor            Lewis Stone            The Patriot  Nominee
...                           ...                    ...      ...
2021 Directing         Chloé Zhao              Nomadland   Winner
     Directing  Thomas Vinterberg          Another Round  Nominee
     Directing      David Fincher                   Mank  Nominee
     Directing    Lee Isaac Chung                 Minari  Nominee
     Directing    Emerald Fennell  Promising Young Woman  Nominee

[1372 rows x 3 columns]